# Check your background of numerical programming

This notebook is a quick recap of the main practical concepts of numerical programming, and introduces the Casadi module of the Pinocchio library:
- Pinocchio is obviously a library gathering the main rigid-body dynamics library.
- Casadi is an auto-differentiation tools, i.e. you can write symbolic expression and differentiate them. It is also wrapping the IpOpt solver, which is a quite robust and generic constraint-based optimization solver. You can then quickly write a problem with some symbolic expression for the cost and constraints, and Casadi will transfer it to IpOpt for solving it.
- The Pinocchio/Casadi module is a part of the Pinocchio solver written by Justin Carpentier which uses Casadi to unroll the algorithms of Pinocchio into symbolic expression, which then can be differentiated. It is a very powerful tool for initiation and prototyping, although it cannot scales to very large problems (in particular whole-body optimization problems) due to various technical problems that we can discuss.

## Definition of the toy problem studied here

We propose to explain how to define and solve inverse-geometry problems, i.e. problems where the main decision variable is a single robot configuration.

Inverse geometry is the problem of computing a configuration $q$ of a robot model that satisfies a set of 'tasks', which we formalize as error functions $e(q)$ that are zero when the task is achieved.

- $q \in \mathcal{C}$: configuration of the robot.
- $e_i(q) \in \mathbb{R}$: error function for task $i$. In this tutorial our error function is real valued.

Typical tasks for limbed robots include putting a foot/end-effector on a surface, moving the center of mass (CoM) to a target location, etc. For example, an error function for a foot position task could be:

$$
e(q) = \|p_\mathrm{foot}^* - p_\mathrm{foot}(q)\|
$$

In this tutorial, we will define a task function $e(q)$ for the end-effector pose of a manipulator and solve the inverse geometry problem

$$
\underset{q}{\mathrm{minimize}}\ e(q)
$$

using an off-the-shelf nonlinear optimizer that evaluates gradients by finite differences (slower), then a more efficient formulation using CasADi (faster).

## Before we start...

%load is a "magic" command of Jupyter. When we will invite you to think to a solution by yourselves, we will indicate it by the %do_not_load command. Feel free to transform it to a plain %load if you want the answer. This is define in the following import.

In [ ]:
import gepetuto.magic

Let's import everything we will need in this tutorial:

In [ ]:
# %load tp1/generated/invgeom_imports
import time
import unittest
import example_robot_data as robex
import numpy as np
import casadi
import pinocchio as pin
import pinocchio.casadi as cpin
from wan2024.meshcat_viewer_wrapper import MeshcatVisualizer, colors


## Set up

Let us start by loading the UR5 robot model from `example-robot-data` and define its neutral configuration as follows:

In [ ]:
# %load tp1/generated/invgeom_robot
robot = robex.load("ur10")
model = robot.model
data = robot.data


We will also use the MeshCat visualizer to watch the optimization unfold. Let's set up a visualizer for our robot model:

In [ ]:
# %load tp1/generated/invgeom_visualizer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)


In [ ]:
viz.display(robot.q0*3)

In [ ]:
viz.viewer.jupyter_cell()

## Definition of the problem

### Task target

Our task will be to move the end effector *frame* of the UR5 to a desired pose (pose := position + orientation). For this purpose, we define:

- The name of the corresponding frame, which is ``"tool0"`` for the UR5.
- The target pose of this frame, which we define by an affine transform ${}_{world} M_{target} \in SE(3)$ (also known as a "placement").

The robot will start from an arbitrary initial configuration (hint: to make the solver's life easier, avoid the singular $[0\ 0\ \ldots\ 0]$ configuration) $q_0$.

In [ ]:
# %load tp1/generated/invgeom_task_params
robot.q0 = np.array([0, -np.pi / 2, 0, 0, 0, 0])

tool_id = model.getFrameId("tool0")

in_world_M_target = pin.SE3(
    pin.utils.rotate("x", np.pi / 4),
    np.array([-0.5, 0.1, 0.2]),
)


### Sidetrack: visualization



In [ ]:
# %load tp1/generated/invgeom_visualizer_callback
# --- Add box to represent target
# Add a vizualization for the target
boxID = "world/box"
viz.addBox(boxID, [0.05, 0.1, 0.2], [1.0, 0.2, 0.2, 0.5])
# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [0.08] * 3, [0.2, 0.2, 1.0, 0.5])

def displayScene(q):
    """
    Given the robot configuration, display:
    - the robot
    - a box representing tool_id
    - a box representing in_world_M_target
    """
    pin.framesForwardKinematics(model, data, q)
    in_world_M_tool = data.oMf[tool_id]
    viz.applyConfiguration(boxID, in_world_M_target)
    viz.applyConfiguration(tipID, in_world_M_tool)
    viz.display(q)
    time.sleep(1e-1)


We are now ready to optimize for real!

## Introducting Casadi

We wil now define our problem in [CasADi](https://web.casadi.org), a framework that allows us to formulate problems as computation graphs from which it can compute derivatives (notably first-order derivatives, *a.k.a.* gradients, and second-order derivatives, *a.k.a.* Hessians) automatically. This feature is called *automatic differentiation*.

### Modelling with CasADi

Pinocchio, in the version 3 we are using in today's tutorial, provides a `pinocchio.casadi` interface to represent configurations (such as $q \in \mathcal{C}$ or $M \in SE(3)$) in CasADi computation graphs, and differentiate them appropriately.

CasADi being designed for Euclidean variables, differentiating properly over manifolds is not as straightforward as representing everything as matrices and vectors in $\mathbb{R}^n$ and taking their usual derivatives. Pinocchio allows us to take derivatives over manifolds properly, without worrying about what happens below the surface. To be able to do that, we first need to redefine our model as a CasADi one:

In [ ]:
# %load tp1/generated/invgeom_casadi_model
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()


Our configuration variable $q$ is now a CasADi ``SX`` symbol of size $n_q$:

In [ ]:
# %load tp1/generated/invgeom_cq
cq = casadi.SX.sym("q", model.nq, 1)


We complete the computation graph from ``cmodel`` to ``cq`` by calling the corresponding forward-kinematics function:

In [ ]:
# %load tp1/generated/invgeom_casadi_fk
cpin.framesForwardKinematics(cmodel, cdata, cq)


In [ ]:
cdata.oMf[tool_id]

### Task error

For a given robot configuration $q$, the end effector will be located at ${}_{world} M_{tool}(q)$, which is a function of $q$ we can compute with Pinocchio's forward kinematics function. The goal of our task will be to make ${}_{world} M_{tool}(q)$ coincide with ${}_{world} M_{target}$, which we can define mathematically as: minimize the (Lie-group) difference ${}_{world} M_{target} \ominus {}_{world} M_{tool}(q)$, or equivalently:

$$
\underset{q}{\mathrm{minimize}} \ e(q) := \| {}_{world} M_{target} \ominus {}_{world} M_{tool}(q) \|
$$

Remember the definition of the $\ominus$ (left) Lie-group difference? No? Let's write it in Python anyway! 

As a CasADi function, it writes:

In [ ]:
# %load tp1/generated/invgeom_casadi_error
error_tool = casadi.Function(
    "etool",
    [cq],
    [
        cpin.log6(
            cdata.oMf[tool_id].inverse() * cpin.SE3(in_world_M_target)
        ).vector
    ],
)


In [ ]:
cpin.log6(cpin.SE3(in_world_M_target)).vector

Finally, we define a computation graph from our optimization variable to our error function:

In [ ]:
# %load tp1/generated/invgeom_casadi_computation_graph
opti = casadi.Opti()
var_q = opti.variable(model.nq)
totalcost = casadi.sumsqr(error_tool(var_q))


In [ ]:
var_q.shape

### IPOPT optimization
We now specify a cost to minimize, and an algorithm to search for the solution.

In [ ]:
# %load tp1/generated/invgeom_ipopt
opti.minimize(totalcost)
opti.solver("ipopt")  # select the backend solver
opti.callback(lambda i: displayScene(opti.debug.value(var_q)))


Finally, we can call the solver:

In [ ]:
# %load tp1/generated/invgeom_solve
# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_q = opti.value(var_q)
except:
    print("ERROR in convergence, plotting debug info.")
    sol_q = opti.debug.value(var_q)


We can check that the solver converged to the same solution:

In [ ]:
# %load tp1/generated/invgeom_check_final_placement
print(
    "The robot finally reached effector placement at\n",
    robot.placement(sol_q, 6),
)


In [ ]:
opti.value(error_tool(var_q))